# LSTM

In [1]:
import jieba
import logging
import numpy as np
from gensim.models import word2vec
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors

Using TensorFlow backend.
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

## jieab tokenize

In [2]:
def jieba_tokenizer(content):
    # load stopwords set
    stopword_set = set()
    with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip(' \n'))

    if str(content)=='nan':
        return ''
    words = jieba.cut(content.replace(' ',''))
    return [word for word in words if word not in stopword_set]

In [4]:
def word2token(word,w2v_model):

    # Retrieve the weights from the model. This is used for initializing the weights
    # in a Keras Embedding layer later
    w2v_weights = w2v_model.wv.vectors
    vocab_size, embedding_size = w2v_weights.shape

    try:
        return w2v_model.wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
    
def token2word(token):
    return w2v_model.wv.index2word[token]

## ckip tokenize

In [ ]:
def ckip_tokenizer(content):
#     if str(content)=='nan':
#         return ''
    word_s = ws([content])
    return word_s

## predict

In [7]:
def suicide_predict(text):
    model = load_model('lstm_model.h5')

    # jieba custom setting.
    jieba.set_dictionary('jieba_dict/dict.txt.big')
    text_token = jieba_tokenizer(text)
    
    
    w2v_model = KeyedVectors.load("w2v_model.wv", mmap='r')   
    
    for i in range(len(text_token)):
        text_token[i] = word2token(text_token[i],w2v_model)

    text_token = pad_sequences([text_token], maxlen=250, padding='post', value=0)

    score = model.predict(text_token)
    if score[0][0] > score[0][1]:
        results = 0
    else:
        results = 1

    return results

In [10]:
suicide_predict("也許雨季來臨前就開始了吧什麼都不想做只覺得好累好累考完駕照雖然開心但更覺得鬆了一口氣終於不用在約定時間跟人見面要讀的paper越積越多進度仍是零雖然可以隨便做做但還是想試著做好想試著做好卻一直無法專心讀著下禮拜要上臺介紹的散文集羨慕那些可以有辦法寫的人而我寫的東西永遠詞不達義--")

Building prefix dict from /home/ubuntu-gpu/Project_Depression/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u156e2db6fd044c50a8c659fc2a66f9b6.cache
Loading model cost 1.295 seconds.
Prefix dict has been built succesfully.
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


0

# BERT

In [2]:
import torch
from torch.utils.data import Dataset,random_split
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
PRETRAINED_MODEL_NAME = "bert-base-chinese"

In [3]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [4]:
model = torch.load('bert_440.h5')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:10} {}".format(name, module))


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout    Dropout(p=0.1, inplace=False)
classifier Linear(in_features=768, out_features=2, bias=True)


In [56]:
text = "我有時會憂鬱、很想死，認為自己沒有活著的價值了，雖然知道自己沒有勇氣真的去自殺，但心中的壓力與眼角的淚水一直冒出…我有試著和其他人談談，找了位要好高中朋友，她在我心中是特別的存在，是我在心裡方面唯一信賴的人，但隨著高中畢業後大家都有自己的生活圈，也不敢像以前那樣有難受就和她講了，好怕打擾到她；有空時我們還是會約出去吃飯什麼的，但這幾個禮拜可能是我太常和她傳訊息了，所以對我不理不睬的，不知道為什麼…我只是想要找人講話而已，對不起。我算慢熟形的怪人，在大學是個系邊，邊到交不到朋友，也不知道怎麼和他們社交，每次在校園內只有我是孤零零一個人，吃飯也是一樣，什麼都是。身為某宗教信徒，曾經在學長姐的邀約下一起去參加此宗教活動，但後來覺得他們只是想要我去參加而已，有被利用的感覺；我也試著參加學校的各類活動希望交到朋友，當以為自己終於突破交到朋友時，只得到他的背叛而已，在大學的第一段友誼居然是這種下場，不知道還能信任誰了。為了讓自己的社交能力看起來沒那麼可悲，在ig上假裝自己就像其他大學同學一樣有朋友、假裝過著正常的日子，成日當個雙面人，實際的朋友圈只有高中的幾個，可是不知道還能維持到什麼時候，唉… 現在除了在社群軟體當雙面人，也只能滑滑手機、試著尋找新興趣填滿空虛的生活了。雖然還有高中朋友，但能信賴和我聊憂鬱的只有那一位，現在她不理我了，我不知道該怎麼辦？現在好想死，可是又沒勇氣，我真的是廢物，什麼都辦不成，只會哭而已，真的好討厭自己。____________________ 更新____________________這幾天終於鼓起勇氣打電話給另一個高中朋友了，雖然在和她訴苦（求救）時還是有所保留，平時不在他人前流淚的我終究忍不住淚腺；電話中對於我雙面人的事實感到驚訝，在外看似開懷的我居然內心如此脆弱。電話另一頭的那位朋友希望我能好好的找人幫助，並鼓勵我提起勇氣找「她」問出了什麼事。我，終於傳了訊息問了…雖然她說是我想太多了，但仍然滅不了心中全部的焦慮，之後她便不讀不回了…唉 對不起是我太煩人了#有現實生活中認出我的不要來指認，我怕自己崩潰"

In [59]:
class_names = ['negative', 'positive']

encoded_text = tokenizer.encode_plus(
    text,
    max_length=300,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [60]:
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)
output = model(input_ids, attention_mask)
_, prediction = torch.max(output[0][0], dim=0)

print(f'Review text: {text}\n')
print(f'Sentiment  : {class_names[prediction]}')

Review text: 我有時會憂鬱、很想死，認為自己沒有活著的價值了，雖然知道自己沒有勇氣真的去自殺，但心中的壓力與眼角的淚水一直冒出…我有試著和其他人談談，找了位要好高中朋友，她在我心中是特別的存在，是我在心裡方面唯一信賴的人，但隨著高中畢業後大家都有自己的生活圈，也不敢像以前那樣有難受就和她講了，好怕打擾到她；有空時我們還是會約出去吃飯什麼的，但這幾個禮拜可能是我太常和她傳訊息了，所以對我不理不睬的，不知道為什麼…我只是想要找人講話而已，對不起。我算慢熟形的怪人，在大學是個系邊，邊到交不到朋友，也不知道怎麼和他們社交，每次在校園內只有我是孤零零一個人，吃飯也是一樣，什麼都是。身為某宗教信徒，曾經在學長姐的邀約下一起去參加此宗教活動，但後來覺得他們只是想要我去參加而已，有被利用的感覺；我也試著參加學校的各類活動希望交到朋友，當以為自己終於突破交到朋友時，只得到他的背叛而已，在大學的第一段友誼居然是這種下場，不知道還能信任誰了。為了讓自己的社交能力看起來沒那麼可悲，在ig上假裝自己就像其他大學同學一樣有朋友、假裝過著正常的日子，成日當個雙面人，實際的朋友圈只有高中的幾個，可是不知道還能維持到什麼時候，唉… 現在除了在社群軟體當雙面人，也只能滑滑手機、試著尋找新興趣填滿空虛的生活了。雖然還有高中朋友，但能信賴和我聊憂鬱的只有那一位，現在她不理我了，我不知道該怎麼辦？現在好想死，可是又沒勇氣，我真的是廢物，什麼都辦不成，只會哭而已，真的好討厭自己。____________________ 更新____________________這幾天終於鼓起勇氣打電話給另一個高中朋友了，雖然在和她訴苦（求救）時還是有所保留，平時不在他人前流淚的我終究忍不住淚腺；電話中對於我雙面人的事實感到驚訝，在外看似開懷的我居然內心如此脆弱。電話另一頭的那位朋友希望我能好好的找人幫助，並鼓勵我提起勇氣找「她」問出了什麼事。我，終於傳了訊息問了…雖然她說是我想太多了，但仍然滅不了心中全部的焦慮，之後她便不讀不回了…唉 對不起是我太煩人了#有現實生活中認出我的不要來指認，我怕自己崩潰

Sentiment  : negative


In [ ]:
本身就有在身心科就診，離下次回診還有一個多禮拜前幾天因為學校有事情要忙，所以那幾天比較累。以前累的時候不會有感覺到情緒起伏很大或異常（就是10分鐘內可以交替亢奮/低落）最近這幾天一直處於嗜睡狀態，一天